In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import tensorflow as tf
from tqdm import tqdm
from glob import glob
from google.colab import drive

import librosa
import librosa.display as dsp
import IPython.display as ipd

warnings.filterwarnings(action='ignore')
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/deep-voice/data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/deep-voice/data


In [2]:
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') #GPU 할당

In [3]:
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(813)

In [4]:
# 저장된 데이터 불러오기
X_train_mel = np.load('X_train_mel.npy')
X_val_mel = np.load('X_val_mel.npy')
y_train = np.load('y_train.npy')
y_val = np.load('y_val.npy')

## VGG19

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, optimizers, metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dropout, Dense, Lambda, Reshape
from tensorflow.keras.applications.vgg19 import preprocess_input

# 입력 차원을 (128, 2970, 1)에서 (128, 128, 3)으로 변경하는 사용자 정의 함수
def preprocess_image(x):
    # 1채널 grayscale 이미지를 3채널 RGB 이미지로 변환
    x = tf.image.grayscale_to_rgb(x)
    # 이미지 크기를 (128, 128)으로 조정
    x = tf.image.resize(x, [128, 128])
    return x

input_shape = (128, 2970, 1)

vgg19 = tf.keras.applications.vgg19.VGG19(
    include_top=False,
    weights='imagenet',
    input_shape=(128, 128, 3)  # input_shape를 (128, 128, 3)으로 변경
)

# VGG19의 모든 계층을 동결합니다.
for layer in vgg19.layers:
    layer.trainable = False

# Sequential 모델을 구성합니다.
model = Sequential([
    Lambda(preprocess_image, input_shape=input_shape),  # 입력 데이터 차원 및 채널 변경 및 크기 조정
    vgg19,
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(2, activation='softmax')  # 최종 출력 계층입니다. 분류하고자 하는 클래스 수에 맞게 조정하세요.
])

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 요약 출력
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 128, 128, 3)       0         
                                                                 
 vgg19 (Functional)          (None, 4, 4, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dropout (Dropout)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 512)               4194816   
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 24,220,226
Trainable params: 4,195,842
Non

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping 콜백 설정
early_stopping = EarlyStopping(
    monitor='val_loss',  # 모니터링할 데이터
    patience=10,         # 성능 향상이 없는 에폭을 몇 번이나 허용할 것인가
    verbose=1,           # Early stopping의 진행 상황을 어떻게 표시할 것인지
    restore_best_weights=True  # 가장 좋은 모델의 가중치를 복원할지 여부
)

# 모델 훈련
history = model.fit(
    X_train_mel, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_mel, y_val),
    callbacks=[early_stopping]  # 콜백 리스트에 early_stopping 추가
)

Epoch 1/10
40/40 [==============================] - 425s 11s/step - loss: 0.3121 - accuracy: 0.9742 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
40/40 [==============================] - 481s 12s/step - loss: 0.0146 - accuracy: 0.9992 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
40/40 [==============================] - 431s 11s/step - loss: 0.0332 - accuracy: 0.9992 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
40/40 [==============================] - 426s 11s/step - loss: 1.5775e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
40/40 [==============================] - 481s 12s/step - loss: 1.0910e-06 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
38/40 [===========================>..] - ETA: 16s - loss: 3.9350e-05 - accuracy: 1.0000

## GoogleNet

In [5]:
import torch
from torchvision import transforms, models
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import ToPILImage
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor

class CustomMelSpectrogramDataset(Dataset):
    def __init__(self, X_data, y_data, transform=None):
        self.X_data = X_data
        self.y_data = y_data
        self.transform = transform

    def __getitem__(self, idx):
        mel_spec = self.X_data[idx]

        # 이미 데이터가 (Height, Width) 형태라고 가정할 때,
        # 여기서는 데이터가 torch.Tensor 형태임을 확인
        if not isinstance(mel_spec, torch.Tensor):
            raise ValueError(f"Expected torch.Tensor, got {type(mel_spec)}.")

        # (높이, 너비, 채널)에서 (채널, 높이, 너비)로 차원 재배열
        mel_spec = mel_spec.permute(2, 0, 1)

        # 멜 스펙트로그램을 PIL 이미지로 변환 시도
        try:
            mel_spec_image = ToPILImage()(mel_spec)
        except ValueError as e:
            # 에러 핸들링
            print(f"Error converting mel spectrogram to image: {e}")
            # 에러 발생 시 특정 처리 로직 (예: 기본 이미지 할당)
            mel_spec_image = None  # or some default handling

        # 추가 변형이 있으면 적용
        if self.transform:
            mel_spec_image = self.transform(mel_spec_image)

        label = self.y_data[idx]
        return mel_spec_image, label

    def __len__(self):
        # 데이터셋 내 아이템의 총 개수 반환
        return len(self.X_data)

# 멜스펙트로그램 데이터를 텐서로 변환
X_train_mel_tensor = torch.tensor(X_train_mel, dtype=torch.float32)
X_val_mel_tensor = torch.tensor(X_val_mel, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# 입력 데이터의 차원 변환 및 채널 조정을 위한 전처리 파이프라인
# 데이터의 크기를 (224, 224)로 조정하고, 1채널을 3채널로 복제
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조정
    transforms.Grayscale(num_output_channels=3), # 1채널을 3채널로 변경
    transforms.ToTensor()
])

# 데이터셋 생성
train_dataset = CustomMelSpectrogramDataset(X_train_mel_tensor, y_train_tensor, transform=transform)
val_dataset = CustomMelSpectrogramDataset(X_val_mel_tensor, y_val_tensor, transform=transform)

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# GoogLeNet 모델 로드
model = models.googlenet(pretrained=True)

# 마지막 FC 레이어를 사용자 정의 데이터셋에 맞게 변경
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# GPU 사용 가능 여부 확인 후 모델을 GPU로 전송
model.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [6]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# DataLoader에서 첫 번째 배치의 형태를 출력합니다.
for batch_idx, (data, target) in enumerate(train_loader):
    print(f'배치 인덱스: {batch_idx}')
    print(f'데이터 형태: {data.shape}')  # 예: torch.Size([32, 10])
    print(f'타겟 형태: {target.shape}')  # 예: torch.Size([32])
    break  # 첫 번째 배치만 출력되도록 합니다.

배치 인덱스: 0
데이터 형태: torch.Size([32, 3, 224, 224])
타겟 형태: torch.Size([32])


In [7]:
# 훈련 과정
num_epochs = 25
for epoch in range(num_epochs):
    model.train()  # 학습 모드
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print('[Epoch %d] loss: %.3f' % (epoch + 1, running_loss))

    # 검증 과정
    model.eval()  # 평가 모드
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('[Epoch %d] val_loss: %.3f, Accuracy: %.2f %%' % (epoch + 1, val_loss, 100 * correct / total))

[Epoch 1] loss: 29.340
[Epoch 1] val_loss: 3.508, Accuracy: 93.75 %
[Epoch 2] loss: 4.167
[Epoch 2] val_loss: 0.454, Accuracy: 99.69 %
[Epoch 3] loss: 2.158
[Epoch 3] val_loss: 0.278, Accuracy: 99.69 %
[Epoch 4] loss: 1.425
[Epoch 4] val_loss: 0.226, Accuracy: 99.69 %
[Epoch 5] loss: 1.152
[Epoch 5] val_loss: 0.159, Accuracy: 100.00 %


KeyboardInterrupt: 